In [2]:
%matplotlib inline 
import seaborn as sns 
import pandas as pd
import numpy as np
def getStatsCallRecord(record_sample,freq='M'):
#     record_sample=call_record[call_record.custorm_id==2427853]
   
    cust_id=record_sample.index.unique()
   
    record_sample.set_index('call_time',inplace=True)
#     record_sample.sort_values('call_time',inplace=True)
    time_stats=lambda x,y:x.groupby('type')['time'].resample(freq,how=y).unstack().T
    
    month_mean_time=time_stats(record_sample,'sum').mean() ## 平均每月来电，拨出，未接通话时长
    
    month_time_cnt=time_stats(record_sample,'count').mean()  ## 平均每月来电，拨出，未接电话频率
    
    values_Of_time_stats=np.concatenate([month_mean_time.values/60.0,[month_mean_time[:2].sum()/60.0],month_time_cnt,[month_time_cnt[:2].sum()] ])
    
    columns_Of_time_stats=['MeanTimeOfAnswer','MeanTimeOfCall','MeanTimeOfMissed','SumTimeOfComm','MeanCntOfAnswer',
                   'MeanCntOfCall','MeanCntOfMissed','SumCntOfComm']
    stats_call_record=pd.DataFrame(data=values_Of_time_stats,index=columns_Of_time_stats,columns=cust_id).T
    return stats_call_record


In [6]:
call_record=pd.read_csv('app_call_record_0301.csv',index_col=0,parse_dates=['call_time'])
# top=call_record.custorm_id.value_counts().index[:3].values
call_record=call_record.set_index('custorm_id')
# call_record=call_record.ix[top,:]
week_freq=['W-MON','W-TUE','W-WED','W-THU','W-FRI','W-SAT','W-SUN']
## 月：M；日历日：D；工作日：B
call_record=call_record[call_record.time!=0]
get_stats_df=lambda freq:call_record.groupby(level=0,group_keys=False).apply(lambda x:getStatsCallRecord(x,freq))  ## 平均每天，每月只需要更改采样频率即可，然后完成整合
Days=get_stats_df('D').add_prefix("Day_")
month=get_stats_df('M').add_prefix("Month_")
hour=get_stats_df('H').add_prefix("Hour_")
MDH=pd.concat([Days,month,hour],axis=1)
for line in  week_freq:
    MDH=pd.concat([MDH,get_stats_df(line).add_prefix(line[2:]+'_')],axis=1)
MDH.to_csv("mdh.csv")

/opt/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


MemoryError: 